In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
PATH_TRAIN = Path('../data/train_data.parquet')

In [3]:
train_data = pd.read_parquet(PATH_TRAIN)

In [4]:
#novo ficheiro para passar categoricas/ordinal.
#remover as colunas que não interessam (muito relacionadas)
#tratar dos valores vazios/outliers
#feature engineering


#check the feature importance

### Drop columns


In [ ]:
columns_to_drop = ['wd_left', 'wd_perc', ]



### Check null values



In [5]:
train_data.isnull().sum()
#give a look at the null values compared to the total number of rows
train_data.isnull().sum() / train_data.shape[0]

brand            0.000000
phase            0.000000
country          0.000000
dayweek          0.000000
month            0.000000
wd_perc          0.000000
ther_area        0.126736
hospital_rate    0.179554
n_nwd_bef        0.000000
n_nwd_aft        0.000000
n_weekday_0      0.000000
n_weekday_1      0.000000
n_weekday_2      0.000000
n_weekday_3      0.000000
n_weekday_4      0.000000
date             0.000000
wd               0.000000
wd_left          0.000000
monthly          0.000000
main_channel     0.179554
dtype: float64

In [6]:
#drop the rows with hospital_rate null values
train_data = train_data.dropna(subset=['hospital_rate'])
train_data.isnull().sum() / train_data.shape[0]


brand            0.0
phase            0.0
country          0.0
dayweek          0.0
month            0.0
wd_perc          0.0
ther_area        0.0
hospital_rate    0.0
n_nwd_bef        0.0
n_nwd_aft        0.0
n_weekday_0      0.0
n_weekday_1      0.0
n_weekday_2      0.0
n_weekday_3      0.0
n_weekday_4      0.0
date             0.0
wd               0.0
wd_left          0.0
monthly          0.0
main_channel     0.0
dtype: float64

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def calculate_anova(df, features, target, k="all"):
  # apply selectKBest to select top features
  best_features = SelectKBest(score_func=f_classif, k=k)
  df_cut = df[features]
  fit = best_features.fit(df_cut, df[target])
  dfscores = pd.DataFrame(fit.scores_)
  dfcolumns = pd.DataFrame(df_cut.columns)

  feature_score = pd.concat([dfcolumns, dfscores], axis=1)
  feature_score.columns = ["Features", "Score"]
  ret = feature_score.sort_values(by="Score", ascending=False)
  return ret.head(k) if k != "all" else ret

anova_features = ['hospital_rate', 'monthly', 'n_weekday_0','n_weekday_1', 'n_weekday_2', 'n_weekday_3', 'n_weekday_4', 'n_nwd_bef', 'n_nwd_aft'];

calculate_anova(train_data, features=anova_features, target='phase')

KeyboardInterrupt: 